In [1]:
import os, random
from pydub import AudioSegment
from xml.dom import minidom
import numpy as np
import pandas as pd
from pysndfx import AudioEffectsChain
from librosa import load
import shutil

In [10]:
f = open("D://Background_noise//meta.txt","r")
contents = f.read()
match_noise_to_type = []
for item in contents.split('	'):
    y = item.split('\naudio/')
    try:
        match_noise_to_type.append((y[1], y[0]))
    except:
        pass

noise_dict = dict(match_noise_to_type)

{'a001_120_150.wav': 'residential_area',
 'a001_150_180.wav': 'residential_area',
 'a001_30_60.wav': 'residential_area',
 'a001_60_90.wav': 'residential_area',
 'a001_90_120.wav': 'residential_area',
 'a003_0_30.wav': 'residential_area',
 'a003_120_150.wav': 'residential_area',
 'a003_150_180.wav': 'residential_area',
 'a003_180_210.wav': 'residential_area',
 'a003_210_240.wav': 'residential_area',
 'a003_30_60.wav': 'residential_area',
 'a003_60_90.wav': 'residential_area',
 'a003_90_120.wav': 'residential_area',
 'a005_0_30.wav': 'residential_area',
 'a005_150_180.wav': 'city_center',
 'a005_210_240.wav': 'city_center',
 'a005_30_60.wav': 'city_center',
 'a005_60_90.wav': 'city_center',
 'a005_90_120.wav': 'city_center',
 'a006_0_30.wav': 'city_center',
 'a006_150_180.wav': 'beach',
 'a006_30_60.wav': 'beach',
 'a006_60_90.wav': 'beach',
 'a006_90_120.wav': 'beach',
 'a009_0_30.wav': 'beach',
 'a009_120_150.wav': 'residential_area',
 'a009_150_180.wav': 'residential_area',
 'a009_180

In [33]:
h_directory = 'D://Datasets//TRAINING//Happy//'
a_directory = 'D://Datasets//TRAINING//Angry//'
n_directory = 'D://Datasets//TRAINING//Neutral//'
s_directory = 'D://Datasets//TRAINING//Sad//'
o_directory = 'D://Datasets//TRAINING//Other//'

h_test = 'D://Datasets//TRAINING//Happy_test//'
a_test = 'D://Datasets//TRAINING//Angry_test//'
n_test = 'D://Datasets//TRAINING//Neutral_test//'
s_test = 'D://Datasets//TRAINING//Sad_test//'
o_test = 'D://Datasets//TRAINING//Other_test//'

h_EMO = 'D://Datasets//EMO-DB//wav//Happy//'
a_EMO = 'D://Datasets//EMO-DB//wav//Angry//'
n_EMO = 'D://Datasets//EMO-DB//wav//Neutral//'
s_EMO = 'D://Datasets//EMO-DB//wav//Sad//'
o_EMO = 'D://Datasets//EMO-DB//wav//Other//'



background_noise = 'D://Background_noise//TUT2016//'
background_noise_test = 'D://Background_noise//noise_test//'

emotion_train_folders = [h_directory, a_directory, n_directory, s_directory, o_directory]
emotion_test_folders = [h_test, a_test, n_test, s_test, o_test]

In [47]:
def rename_emotion_wavs(emotion_directory):
    if 'Happy' in emotion_directory:
        word = 'Happy'
    elif 'Angry' in emotion_directory:
        word = 'Angry'
    elif 'Neutral' in emotion_directory:
        word = 'Neutral'
    elif 'Sad' in emotion_directory:
        word = 'Sad'
    else:
        word = 'Other'
    
    index = 0
    for file in os.listdir(emotion_directory):
        src = emotion_directory + file
        dst = emotion_directory + word + str(index) + '.wav'
        try:
            os.rename(src, dst)
        except WindowsError:
            os.remove(dst)
            os.rename(src, dst)
        index += 1
        print(dst)
        
#rename_emotion_wavs(h_directory)
#rename_emotion_wavs(a_directory)
#rename_emotion_wavs(n_directory)
#rename_emotion_wavs(s_directory)
#rename_emotion_wavs(o_directory)

D://Datasets//TRAINING//Happy//Happy0.wav
D://Datasets//TRAINING//Happy//Happy1.wav
D://Datasets//TRAINING//Happy//Happy2.wav
D://Datasets//TRAINING//Happy//Happy3.wav
D://Datasets//TRAINING//Happy//Happy4.wav
D://Datasets//TRAINING//Happy//Happy5.wav
D://Datasets//TRAINING//Happy//Happy6.wav
D://Datasets//TRAINING//Happy//Happy7.wav
D://Datasets//TRAINING//Happy//Happy8.wav
D://Datasets//TRAINING//Happy//Happy9.wav
D://Datasets//TRAINING//Happy//Happy10.wav
D://Datasets//TRAINING//Happy//Happy11.wav
D://Datasets//TRAINING//Happy//Happy12.wav
D://Datasets//TRAINING//Happy//Happy13.wav
D://Datasets//TRAINING//Happy//Happy14.wav
D://Datasets//TRAINING//Happy//Happy15.wav
D://Datasets//TRAINING//Happy//Happy16.wav
D://Datasets//TRAINING//Happy//Happy17.wav
D://Datasets//TRAINING//Happy//Happy18.wav
D://Datasets//TRAINING//Happy//Happy19.wav
D://Datasets//TRAINING//Happy//Happy20.wav
D://Datasets//TRAINING//Happy//Happy21.wav
D://Datasets//TRAINING//Happy//Happy22.wav
D://Datasets//TRAININ

D://Datasets//TRAINING//Happy//Happy193.wav
D://Datasets//TRAINING//Happy//Happy194.wav
D://Datasets//TRAINING//Happy//Happy195.wav
D://Datasets//TRAINING//Happy//Happy196.wav
D://Datasets//TRAINING//Happy//Happy197.wav
D://Datasets//TRAINING//Happy//Happy198.wav
D://Datasets//TRAINING//Happy//Happy199.wav
D://Datasets//TRAINING//Happy//Happy200.wav
D://Datasets//TRAINING//Happy//Happy201.wav
D://Datasets//TRAINING//Happy//Happy202.wav
D://Datasets//TRAINING//Happy//Happy203.wav
D://Datasets//TRAINING//Happy//Happy204.wav
D://Datasets//TRAINING//Happy//Happy205.wav
D://Datasets//TRAINING//Happy//Happy206.wav
D://Datasets//TRAINING//Happy//Happy207.wav
D://Datasets//TRAINING//Happy//Happy208.wav
D://Datasets//TRAINING//Happy//Happy209.wav
D://Datasets//TRAINING//Happy//Happy210.wav
D://Datasets//TRAINING//Happy//Happy211.wav
D://Datasets//TRAINING//Happy//Happy212.wav
D://Datasets//TRAINING//Happy//Happy213.wav
D://Datasets//TRAINING//Happy//Happy214.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy383.wav
D://Datasets//TRAINING//Happy//Happy384.wav
D://Datasets//TRAINING//Happy//Happy385.wav
D://Datasets//TRAINING//Happy//Happy386.wav
D://Datasets//TRAINING//Happy//Happy387.wav
D://Datasets//TRAINING//Happy//Happy388.wav
D://Datasets//TRAINING//Happy//Happy389.wav
D://Datasets//TRAINING//Happy//Happy390.wav
D://Datasets//TRAINING//Happy//Happy391.wav
D://Datasets//TRAINING//Happy//Happy392.wav
D://Datasets//TRAINING//Happy//Happy393.wav
D://Datasets//TRAINING//Happy//Happy394.wav
D://Datasets//TRAINING//Happy//Happy395.wav
D://Datasets//TRAINING//Happy//Happy396.wav
D://Datasets//TRAINING//Happy//Happy397.wav
D://Datasets//TRAINING//Happy//Happy398.wav
D://Datasets//TRAINING//Happy//Happy399.wav
D://Datasets//TRAINING//Happy//Happy400.wav
D://Datasets//TRAINING//Happy//Happy401.wav
D://Datasets//TRAINING//Happy//Happy402.wav
D://Datasets//TRAINING//Happy//Happy403.wav
D://Datasets//TRAINING//Happy//Happy404.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy570.wav
D://Datasets//TRAINING//Happy//Happy571.wav
D://Datasets//TRAINING//Happy//Happy572.wav
D://Datasets//TRAINING//Happy//Happy573.wav
D://Datasets//TRAINING//Happy//Happy574.wav
D://Datasets//TRAINING//Happy//Happy575.wav
D://Datasets//TRAINING//Happy//Happy576.wav
D://Datasets//TRAINING//Happy//Happy577.wav
D://Datasets//TRAINING//Happy//Happy578.wav
D://Datasets//TRAINING//Happy//Happy579.wav
D://Datasets//TRAINING//Happy//Happy580.wav
D://Datasets//TRAINING//Happy//Happy581.wav
D://Datasets//TRAINING//Happy//Happy582.wav
D://Datasets//TRAINING//Happy//Happy583.wav
D://Datasets//TRAINING//Happy//Happy584.wav
D://Datasets//TRAINING//Happy//Happy585.wav
D://Datasets//TRAINING//Happy//Happy586.wav
D://Datasets//TRAINING//Happy//Happy587.wav
D://Datasets//TRAINING//Happy//Happy588.wav
D://Datasets//TRAINING//Happy//Happy589.wav
D://Datasets//TRAINING//Happy//Happy590.wav
D://Datasets//TRAINING//Happy//Happy591.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy762.wav
D://Datasets//TRAINING//Happy//Happy763.wav
D://Datasets//TRAINING//Happy//Happy764.wav
D://Datasets//TRAINING//Happy//Happy765.wav
D://Datasets//TRAINING//Happy//Happy766.wav
D://Datasets//TRAINING//Happy//Happy767.wav
D://Datasets//TRAINING//Happy//Happy768.wav
D://Datasets//TRAINING//Happy//Happy769.wav
D://Datasets//TRAINING//Happy//Happy770.wav
D://Datasets//TRAINING//Happy//Happy771.wav
D://Datasets//TRAINING//Happy//Happy772.wav
D://Datasets//TRAINING//Happy//Happy773.wav
D://Datasets//TRAINING//Happy//Happy774.wav
D://Datasets//TRAINING//Happy//Happy775.wav
D://Datasets//TRAINING//Happy//Happy776.wav
D://Datasets//TRAINING//Happy//Happy777.wav
D://Datasets//TRAINING//Happy//Happy778.wav
D://Datasets//TRAINING//Happy//Happy779.wav
D://Datasets//TRAINING//Happy//Happy780.wav
D://Datasets//TRAINING//Happy//Happy781.wav
D://Datasets//TRAINING//Happy//Happy782.wav
D://Datasets//TRAINING//Happy//Happy783.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy952.wav
D://Datasets//TRAINING//Happy//Happy953.wav
D://Datasets//TRAINING//Happy//Happy954.wav
D://Datasets//TRAINING//Happy//Happy955.wav
D://Datasets//TRAINING//Happy//Happy956.wav
D://Datasets//TRAINING//Happy//Happy957.wav
D://Datasets//TRAINING//Happy//Happy958.wav
D://Datasets//TRAINING//Happy//Happy959.wav
D://Datasets//TRAINING//Happy//Happy960.wav
D://Datasets//TRAINING//Happy//Happy961.wav
D://Datasets//TRAINING//Happy//Happy962.wav
D://Datasets//TRAINING//Happy//Happy963.wav
D://Datasets//TRAINING//Happy//Happy964.wav
D://Datasets//TRAINING//Happy//Happy965.wav
D://Datasets//TRAINING//Happy//Happy966.wav
D://Datasets//TRAINING//Happy//Happy967.wav
D://Datasets//TRAINING//Happy//Happy968.wav
D://Datasets//TRAINING//Happy//Happy969.wav
D://Datasets//TRAINING//Happy//Happy970.wav
D://Datasets//TRAINING//Happy//Happy971.wav
D://Datasets//TRAINING//Happy//Happy972.wav
D://Datasets//TRAINING//Happy//Happy973.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy1142.wav
D://Datasets//TRAINING//Happy//Happy1143.wav
D://Datasets//TRAINING//Happy//Happy1144.wav
D://Datasets//TRAINING//Happy//Happy1145.wav
D://Datasets//TRAINING//Happy//Happy1146.wav
D://Datasets//TRAINING//Happy//Happy1147.wav
D://Datasets//TRAINING//Happy//Happy1148.wav
D://Datasets//TRAINING//Happy//Happy1149.wav
D://Datasets//TRAINING//Happy//Happy1150.wav
D://Datasets//TRAINING//Happy//Happy1151.wav
D://Datasets//TRAINING//Happy//Happy1152.wav
D://Datasets//TRAINING//Happy//Happy1153.wav
D://Datasets//TRAINING//Happy//Happy1154.wav
D://Datasets//TRAINING//Happy//Happy1155.wav
D://Datasets//TRAINING//Happy//Happy1156.wav
D://Datasets//TRAINING//Happy//Happy1157.wav
D://Datasets//TRAINING//Happy//Happy1158.wav
D://Datasets//TRAINING//Happy//Happy1159.wav
D://Datasets//TRAINING//Happy//Happy1160.wav
D://Datasets//TRAINING//Happy//Happy1161.wav
D://Datasets//TRAINING//Happy//Happy1162.wav
D://Datasets//TRAINING//Happy//Happy1163.wav
D://Datase

D://Datasets//TRAINING//Happy//Happy1326.wav
D://Datasets//TRAINING//Happy//Happy1327.wav
D://Datasets//TRAINING//Happy//Happy1328.wav
D://Datasets//TRAINING//Happy//Happy1329.wav
D://Datasets//TRAINING//Happy//Happy1330.wav
D://Datasets//TRAINING//Happy//Happy1331.wav
D://Datasets//TRAINING//Happy//Happy1332.wav
D://Datasets//TRAINING//Happy//Happy1333.wav
D://Datasets//TRAINING//Happy//Happy1334.wav
D://Datasets//TRAINING//Happy//Happy1335.wav
D://Datasets//TRAINING//Happy//Happy1336.wav
D://Datasets//TRAINING//Happy//Happy1337.wav
D://Datasets//TRAINING//Happy//Happy1338.wav
D://Datasets//TRAINING//Happy//Happy1339.wav
D://Datasets//TRAINING//Happy//Happy1340.wav
D://Datasets//TRAINING//Happy//Happy1341.wav
D://Datasets//TRAINING//Happy//Happy1342.wav
D://Datasets//TRAINING//Happy//Happy1343.wav
D://Datasets//TRAINING//Happy//Happy1344.wav
D://Datasets//TRAINING//Happy//Happy1345.wav
D://Datasets//TRAINING//Happy//Happy1346.wav
D://Datasets//TRAINING//Happy//Happy1347.wav
D://Datase

D://Datasets//TRAINING//Happy//Happy1513.wav
D://Datasets//TRAINING//Happy//Happy1514.wav
D://Datasets//TRAINING//Happy//Happy1515.wav
D://Datasets//TRAINING//Happy//Happy1516.wav
D://Datasets//TRAINING//Happy//Happy1517.wav
D://Datasets//TRAINING//Happy//Happy1518.wav
D://Datasets//TRAINING//Happy//Happy1519.wav
D://Datasets//TRAINING//Happy//Happy1520.wav
D://Datasets//TRAINING//Happy//Happy1521.wav
D://Datasets//TRAINING//Happy//Happy1522.wav
D://Datasets//TRAINING//Happy//Happy1523.wav
D://Datasets//TRAINING//Happy//Happy1524.wav
D://Datasets//TRAINING//Happy//Happy1525.wav
D://Datasets//TRAINING//Happy//Happy1526.wav
D://Datasets//TRAINING//Happy//Happy1527.wav
D://Datasets//TRAINING//Happy//Happy1528.wav
D://Datasets//TRAINING//Happy//Happy1529.wav
D://Datasets//TRAINING//Happy//Happy1530.wav
D://Datasets//TRAINING//Happy//Happy1531.wav
D://Datasets//TRAINING//Happy//Happy1532.wav
D://Datasets//TRAINING//Happy//Happy1533.wav
D://Datasets//TRAINING//Happy//Happy1534.wav
D://Datase

D://Datasets//TRAINING//Happy//Happy1696.wav
D://Datasets//TRAINING//Happy//Happy1697.wav
D://Datasets//TRAINING//Happy//Happy1698.wav
D://Datasets//TRAINING//Happy//Happy1699.wav
D://Datasets//TRAINING//Happy//Happy1700.wav
D://Datasets//TRAINING//Happy//Happy1701.wav
D://Datasets//TRAINING//Happy//Happy1702.wav
D://Datasets//TRAINING//Happy//Happy1703.wav
D://Datasets//TRAINING//Happy//Happy1704.wav
D://Datasets//TRAINING//Happy//Happy1705.wav
D://Datasets//TRAINING//Happy//Happy1706.wav
D://Datasets//TRAINING//Happy//Happy1707.wav
D://Datasets//TRAINING//Happy//Happy1708.wav
D://Datasets//TRAINING//Happy//Happy1709.wav
D://Datasets//TRAINING//Happy//Happy1710.wav
D://Datasets//TRAINING//Happy//Happy1711.wav
D://Datasets//TRAINING//Happy//Happy1712.wav
D://Datasets//TRAINING//Happy//Happy1713.wav
D://Datasets//TRAINING//Happy//Happy1714.wav
D://Datasets//TRAINING//Happy//Happy1715.wav
D://Datasets//TRAINING//Happy//Happy1716.wav
D://Datasets//TRAINING//Happy//Happy1717.wav
D://Datase

In [22]:
def rename_noise_wavs(noise_directory):
    index = 0
    for noise in os.listdir(noise_directory):
        try:
            noise_type = noise_dict.get(noise)
            new_noise_name = noise_type + '_' + str(index)
            src = noise_directory + noise
            dst = noise_directory + new_noise_name + '.wav'
            os.rename(src, dst) 
            index += 1
            
            print(dst)
        except:
            pass

# rename_noise_wavs(background_noise_test)
# rename_noise_wavs(background_noise)

D://Background_noise//noise_test//residential_area_0.wav
D://Background_noise//noise_test//residential_area_1.wav
D://Background_noise//noise_test//residential_area_2.wav
D://Background_noise//noise_test//residential_area_3.wav
D://Background_noise//noise_test//residential_area_4.wav
D://Background_noise//noise_test//residential_area_5.wav
D://Background_noise//noise_test//residential_area_6.wav
D://Background_noise//noise_test//residential_area_7.wav
D://Background_noise//noise_test//residential_area_8.wav
D://Background_noise//noise_test//beach_9.wav
D://Background_noise//noise_test//beach_10.wav
D://Background_noise//noise_test//beach_11.wav
D://Background_noise//noise_test//beach_12.wav
D://Background_noise//noise_test//park_13.wav
D://Background_noise//noise_test//park_14.wav
D://Background_noise//noise_test//park_15.wav
D://Background_noise//noise_test//park_16.wav
D://Background_noise//noise_test//park_17.wav
D://Background_noise//noise_test//park_18.wav
D://Background_noise//no

D://Background_noise//noise_test//grocery_store_171.wav
D://Background_noise//noise_test//grocery_store_172.wav
D://Background_noise//noise_test//grocery_store_173.wav
D://Background_noise//noise_test//car_174.wav
D://Background_noise//noise_test//city_center_175.wav
D://Background_noise//noise_test//city_center_176.wav
D://Background_noise//noise_test//city_center_177.wav
D://Background_noise//noise_test//city_center_178.wav
D://Background_noise//noise_test//city_center_179.wav
D://Background_noise//noise_test//car_180.wav
D://Background_noise//noise_test//car_181.wav
D://Background_noise//noise_test//car_182.wav
D://Background_noise//noise_test//car_183.wav
D://Background_noise//noise_test//city_center_184.wav
D://Background_noise//noise_test//city_center_185.wav
D://Background_noise//noise_test//city_center_186.wav
D://Background_noise//noise_test//city_center_187.wav
D://Background_noise//noise_test//grocery_store_188.wav
D://Background_noise//noise_test//bus_189.wav
D://Background

D://Background_noise//TUT2016//forest_path_156.wav
D://Background_noise//TUT2016//forest_path_157.wav
D://Background_noise//TUT2016//forest_path_158.wav
D://Background_noise//TUT2016//forest_path_159.wav
D://Background_noise//TUT2016//forest_path_160.wav
D://Background_noise//TUT2016//forest_path_161.wav
D://Background_noise//TUT2016//forest_path_162.wav
D://Background_noise//TUT2016//forest_path_163.wav
D://Background_noise//TUT2016//forest_path_164.wav
D://Background_noise//TUT2016//car_165.wav
D://Background_noise//TUT2016//car_166.wav
D://Background_noise//TUT2016//car_167.wav
D://Background_noise//TUT2016//car_168.wav
D://Background_noise//TUT2016//car_169.wav
D://Background_noise//TUT2016//car_170.wav
D://Background_noise//TUT2016//forest_path_171.wav
D://Background_noise//TUT2016//forest_path_172.wav
D://Background_noise//TUT2016//forest_path_173.wav
D://Background_noise//TUT2016//forest_path_174.wav
D://Background_noise//TUT2016//forest_path_175.wav
D://Background_noise//TUT201

D://Background_noise//TUT2016//city_center_339.wav
D://Background_noise//TUT2016//city_center_340.wav
D://Background_noise//TUT2016//car_341.wav
D://Background_noise//TUT2016//car_342.wav
D://Background_noise//TUT2016//car_343.wav
D://Background_noise//TUT2016//car_344.wav
D://Background_noise//TUT2016//car_345.wav
D://Background_noise//TUT2016//car_346.wav
D://Background_noise//TUT2016//car_347.wav
D://Background_noise//TUT2016//car_348.wav
D://Background_noise//TUT2016//car_349.wav
D://Background_noise//TUT2016//car_350.wav
D://Background_noise//TUT2016//car_351.wav
D://Background_noise//TUT2016//car_352.wav
D://Background_noise//TUT2016//bus_353.wav
D://Background_noise//TUT2016//bus_354.wav
D://Background_noise//TUT2016//bus_355.wav
D://Background_noise//TUT2016//bus_356.wav
D://Background_noise//TUT2016//bus_357.wav
D://Background_noise//TUT2016//bus_358.wav
D://Background_noise//TUT2016//bus_359.wav
D://Background_noise//TUT2016//bus_360.wav
D://Background_noise//TUT2016//train_3

D://Background_noise//TUT2016//beach_516.wav
D://Background_noise//TUT2016//beach_517.wav
D://Background_noise//TUT2016//beach_518.wav
D://Background_noise//TUT2016//beach_519.wav
D://Background_noise//TUT2016//beach_520.wav
D://Background_noise//TUT2016//beach_521.wav
D://Background_noise//TUT2016//beach_522.wav
D://Background_noise//TUT2016//beach_523.wav
D://Background_noise//TUT2016//park_524.wav
D://Background_noise//TUT2016//park_525.wav
D://Background_noise//TUT2016//park_526.wav
D://Background_noise//TUT2016//park_527.wav
D://Background_noise//TUT2016//park_528.wav
D://Background_noise//TUT2016//library_529.wav
D://Background_noise//TUT2016//library_530.wav
D://Background_noise//TUT2016//library_531.wav
D://Background_noise//TUT2016//library_532.wav
D://Background_noise//TUT2016//library_533.wav
D://Background_noise//TUT2016//home_534.wav
D://Background_noise//TUT2016//home_535.wav
D://Background_noise//TUT2016//home_536.wav
D://Background_noise//TUT2016//home_537.wav
D://Backg

D://Background_noise//TUT2016//metro_station_704.wav
D://Background_noise//TUT2016//metro_station_705.wav
D://Background_noise//TUT2016//metro_station_706.wav
D://Background_noise//TUT2016//metro_station_707.wav
D://Background_noise//TUT2016//metro_station_708.wav
D://Background_noise//TUT2016//metro_station_709.wav
D://Background_noise//TUT2016//metro_station_710.wav
D://Background_noise//TUT2016//metro_station_711.wav
D://Background_noise//TUT2016//metro_station_712.wav
D://Background_noise//TUT2016//metro_station_713.wav
D://Background_noise//TUT2016//metro_station_714.wav
D://Background_noise//TUT2016//metro_station_715.wav
D://Background_noise//TUT2016//metro_station_716.wav
D://Background_noise//TUT2016//metro_station_717.wav
D://Background_noise//TUT2016//metro_station_718.wav
D://Background_noise//TUT2016//metro_station_719.wav
D://Background_noise//TUT2016//metro_station_720.wav
D://Background_noise//TUT2016//tram_721.wav
D://Background_noise//TUT2016//tram_722.wav
D://Backgro

D://Background_noise//TUT2016//office_888.wav
D://Background_noise//TUT2016//office_889.wav
D://Background_noise//TUT2016//office_890.wav
D://Background_noise//TUT2016//office_891.wav


In [30]:
def delete_directory(path):
    shutil.rmtree(path, ignore_errors=True)

In [31]:
def partition_directory(path, new_path, percent):
    
    if not os.path.exists(new_path):
        os.mkdir(new_path)
    else:
        delete_directory(new_path)
        os.makdir(new_path)
        
    total = len(os.listdir(path))
    test_number = int(total * percent)
    test_set = np.random.permutation(total)[0:test_number]
    
    for index in test_set:
        try:
            if os.listdir(path)[index].endswith('.wav'):
                original_location = path + os.listdir(path)[index]
                new_location = new_path + os.listdir(path)[index]
                os.rename(original_location, new_location)
                print(new_location)
        except:
            pass

    
    print('# of training data + # of testing data = ' + str(total))
    print(str(percent) + ' of the data is in training set')
    print('# of training data = ' + str(test_set.shape))


In [48]:
percent = 0.2
partition_directory(h_directory, h_test, percent)
#partition_directory(a_directory, a_test, percent)
#partition_directory(n_directory, n_test, percent)
#partition_directory(s_directory, s_test, percent)
#partition_directory(o_directory, o_test, percent)

D://Datasets//TRAINING//Happy_test//Happy1291.wav
D://Datasets//TRAINING//Happy_test//Happy691.wav
D://Datasets//TRAINING//Happy_test//Happy1303.wav
D://Datasets//TRAINING//Happy_test//Happy1491.wav
D://Datasets//TRAINING//Happy_test//Happy245.wav
D://Datasets//TRAINING//Happy_test//Happy1506.wav
D://Datasets//TRAINING//Happy_test//Happy1779.wav
D://Datasets//TRAINING//Happy_test//Happy835.wav
D://Datasets//TRAINING//Happy_test//Happy93.wav
D://Datasets//TRAINING//Happy_test//Happy569.wav
D://Datasets//TRAINING//Happy_test//Happy994.wav
D://Datasets//TRAINING//Happy_test//Happy154.wav
D://Datasets//TRAINING//Happy_test//Happy290.wav
D://Datasets//TRAINING//Happy_test//Happy1137.wav
D://Datasets//TRAINING//Happy_test//Happy1231.wav
D://Datasets//TRAINING//Happy_test//Happy228.wav
D://Datasets//TRAINING//Happy_test//Happy67.wav
D://Datasets//TRAINING//Happy_test//Happy1102.wav
D://Datasets//TRAINING//Happy_test//Happy695.wav
D://Datasets//TRAINING//Happy_test//Happy1.wav
D://Datasets//TR

D://Datasets//TRAINING//Happy_test//Happy253.wav
D://Datasets//TRAINING//Happy_test//Happy1603.wav
D://Datasets//TRAINING//Happy_test//Happy1185.wav
D://Datasets//TRAINING//Happy_test//Happy929.wav
D://Datasets//TRAINING//Happy_test//Happy135.wav
D://Datasets//TRAINING//Happy_test//Happy1672.wav
D://Datasets//TRAINING//Happy_test//Happy1145.wav
D://Datasets//TRAINING//Happy_test//Happy192.wav
D://Datasets//TRAINING//Happy_test//Happy1056.wav
D://Datasets//TRAINING//Happy_test//Happy581.wav
D://Datasets//TRAINING//Happy_test//Happy729.wav
D://Datasets//TRAINING//Happy_test//Happy1284.wav
D://Datasets//TRAINING//Happy_test//Happy1165.wav
D://Datasets//TRAINING//Happy_test//Happy4.wav
D://Datasets//TRAINING//Happy_test//Happy1788.wav
D://Datasets//TRAINING//Happy_test//Happy1071.wav
D://Datasets//TRAINING//Happy_test//Happy26.wav
D://Datasets//TRAINING//Happy_test//Happy1638.wav
D://Datasets//TRAINING//Happy_test//Happy872.wav
D://Datasets//TRAINING//Happy_test//Happy272.wav
D://Datasets/

In [49]:
def pad_silence_per_file(emotionfile, newSoundFile):
    
    emotionsound = AudioSegment.from_wav(emotionfile)
    file_duration = emotionsound.duration_seconds * 1000
    silence = AudioSegment.silent(duration=5000)
    threshold = 5000 - file_duration
    if threshold > 0:
        overlay_start = np.random.randint(0, threshold)
    else:
        emotionsound = emotionsound[0:5000]
        overlay_start = 0
    
    newSound = silence.overlay(emotionsound, position=overlay_start)
    
    newSound=newSound[0:5000]
    newSound.export(newSoundFile, format='wav')  ### save the new generated file in a folder

In [50]:
def pad_silence_per_folder(directory, extension):
    for file in os.listdir(directory):
        if file.endswith(extension) and file[0] != '.':
            newSoundFile = directory + file
            pad_silence_per_file(newSoundFile, newSoundFile)
            print(newSoundFile)

In [51]:
'''
for path in emotion_train_folders:
    pad_silence_per_folder(path, '.wav')
    
for path in emotion_test_folders:
    pad_silence_per_folder(path, '.wav')
'''
#pad_silence_per_folder(h_directory, '.wav')
#pad_silence_per_folder(h_test, '.wav')

D://Datasets//TRAINING//Happy//Happy374.wav
D://Datasets//TRAINING//Happy//Happy375.wav
D://Datasets//TRAINING//Happy//Happy376.wav
D://Datasets//TRAINING//Happy//Happy377.wav
D://Datasets//TRAINING//Happy//Happy378.wav
D://Datasets//TRAINING//Happy//Happy379.wav
D://Datasets//TRAINING//Happy//Happy380.wav
D://Datasets//TRAINING//Happy//Happy381.wav
D://Datasets//TRAINING//Happy//Happy382.wav
D://Datasets//TRAINING//Happy//Happy383.wav
D://Datasets//TRAINING//Happy//Happy384.wav
D://Datasets//TRAINING//Happy//Happy385.wav
D://Datasets//TRAINING//Happy//Happy386.wav
D://Datasets//TRAINING//Happy//Happy388.wav
D://Datasets//TRAINING//Happy//Happy390.wav
D://Datasets//TRAINING//Happy//Happy391.wav
D://Datasets//TRAINING//Happy//Happy392.wav
D://Datasets//TRAINING//Happy//Happy393.wav
D://Datasets//TRAINING//Happy//Happy394.wav
D://Datasets//TRAINING//Happy//Happy395.wav
D://Datasets//TRAINING//Happy//Happy396.wav
D://Datasets//TRAINING//Happy//Happy397.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy603.wav
D://Datasets//TRAINING//Happy//Happy171.wav
D://Datasets//TRAINING//Happy//Happy605.wav
D://Datasets//TRAINING//Happy//Happy606.wav
D://Datasets//TRAINING//Happy//Happy608.wav
D://Datasets//TRAINING//Happy//Happy609.wav
D://Datasets//TRAINING//Happy//Happy610.wav
D://Datasets//TRAINING//Happy//Happy177.wav
D://Datasets//TRAINING//Happy//Happy612.wav
D://Datasets//TRAINING//Happy//Happy613.wav
D://Datasets//TRAINING//Happy//Happy614.wav
D://Datasets//TRAINING//Happy//Happy615.wav
D://Datasets//TRAINING//Happy//Happy617.wav
D://Datasets//TRAINING//Happy//Happy619.wav
D://Datasets//TRAINING//Happy//Happy620.wav
D://Datasets//TRAINING//Happy//Happy621.wav
D://Datasets//TRAINING//Happy//Happy622.wav
D://Datasets//TRAINING//Happy//Happy623.wav
D://Datasets//TRAINING//Happy//Happy624.wav
D://Datasets//TRAINING//Happy//Happy625.wav
D://Datasets//TRAINING//Happy//Happy626.wav
D://Datasets//TRAINING//Happy//Happy627.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy829.wav
D://Datasets//TRAINING//Happy//Happy830.wav
D://Datasets//TRAINING//Happy//Happy831.wav
D://Datasets//TRAINING//Happy//Happy832.wav
D://Datasets//TRAINING//Happy//Happy833.wav
D://Datasets//TRAINING//Happy//Happy837.wav
D://Datasets//TRAINING//Happy//Happy838.wav
D://Datasets//TRAINING//Happy//Happy839.wav
D://Datasets//TRAINING//Happy//Happy842.wav
D://Datasets//TRAINING//Happy//Happy843.wav
D://Datasets//TRAINING//Happy//Happy844.wav
D://Datasets//TRAINING//Happy//Happy845.wav
D://Datasets//TRAINING//Happy//Happy846.wav
D://Datasets//TRAINING//Happy//Happy847.wav
D://Datasets//TRAINING//Happy//Happy848.wav
D://Datasets//TRAINING//Happy//Happy849.wav
D://Datasets//TRAINING//Happy//Happy850.wav
D://Datasets//TRAINING//Happy//Happy851.wav
D://Datasets//TRAINING//Happy//Happy852.wav
D://Datasets//TRAINING//Happy//Happy853.wav
D://Datasets//TRAINING//Happy//Happy854.wav
D://Datasets//TRAINING//Happy//Happy855.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy//Happy1049.wav
D://Datasets//TRAINING//Happy//Happy1052.wav
D://Datasets//TRAINING//Happy//Happy1053.wav
D://Datasets//TRAINING//Happy//Happy1054.wav
D://Datasets//TRAINING//Happy//Happy1055.wav
D://Datasets//TRAINING//Happy//Happy1057.wav
D://Datasets//TRAINING//Happy//Happy1058.wav
D://Datasets//TRAINING//Happy//Happy1059.wav
D://Datasets//TRAINING//Happy//Happy1060.wav
D://Datasets//TRAINING//Happy//Happy1061.wav
D://Datasets//TRAINING//Happy//Happy1062.wav
D://Datasets//TRAINING//Happy//Happy1063.wav
D://Datasets//TRAINING//Happy//Happy1064.wav
D://Datasets//TRAINING//Happy//Happy1065.wav
D://Datasets//TRAINING//Happy//Happy1066.wav
D://Datasets//TRAINING//Happy//Happy1067.wav
D://Datasets//TRAINING//Happy//Happy1069.wav
D://Datasets//TRAINING//Happy//Happy1070.wav
D://Datasets//TRAINING//Happy//Happy1072.wav
D://Datasets//TRAINING//Happy//Happy1073.wav
D://Datasets//TRAINING//Happy//Happy1074.wav
D://Datasets//TRAINING//Happy//Happy1076.wav
D://Datase

D://Datasets//TRAINING//Happy//Happy1279.wav
D://Datasets//TRAINING//Happy//Happy1280.wav
D://Datasets//TRAINING//Happy//Happy1281.wav
D://Datasets//TRAINING//Happy//Happy1282.wav
D://Datasets//TRAINING//Happy//Happy1285.wav
D://Datasets//TRAINING//Happy//Happy700.wav
D://Datasets//TRAINING//Happy//Happy1286.wav
D://Datasets//TRAINING//Happy//Happy1287.wav
D://Datasets//TRAINING//Happy//Happy1288.wav
D://Datasets//TRAINING//Happy//Happy1289.wav
D://Datasets//TRAINING//Happy//Happy1292.wav
D://Datasets//TRAINING//Happy//Happy1293.wav
D://Datasets//TRAINING//Happy//Happy1294.wav
D://Datasets//TRAINING//Happy//Happy1295.wav
D://Datasets//TRAINING//Happy//Happy1296.wav
D://Datasets//TRAINING//Happy//Happy1297.wav
D://Datasets//TRAINING//Happy//Happy1298.wav
D://Datasets//TRAINING//Happy//Happy711.wav
D://Datasets//TRAINING//Happy//Happy1299.wav
D://Datasets//TRAINING//Happy//Happy1300.wav
D://Datasets//TRAINING//Happy//Happy1301.wav
D://Datasets//TRAINING//Happy//Happy1302.wav
D://Datasets

D://Datasets//TRAINING//Happy//Happy1504.wav
D://Datasets//TRAINING//Happy//Happy1505.wav
D://Datasets//TRAINING//Happy//Happy1507.wav
D://Datasets//TRAINING//Happy//Happy1508.wav
D://Datasets//TRAINING//Happy//Happy871.wav
D://Datasets//TRAINING//Happy//Happy1509.wav
D://Datasets//TRAINING//Happy//Happy1510.wav
D://Datasets//TRAINING//Happy//Happy1512.wav
D://Datasets//TRAINING//Happy//Happy1513.wav
D://Datasets//TRAINING//Happy//Happy1514.wav
D://Datasets//TRAINING//Happy//Happy1516.wav
D://Datasets//TRAINING//Happy//Happy1517.wav
D://Datasets//TRAINING//Happy//Happy1519.wav
D://Datasets//TRAINING//Happy//Happy1520.wav
D://Datasets//TRAINING//Happy//Happy1521.wav
D://Datasets//TRAINING//Happy//Happy1522.wav
D://Datasets//TRAINING//Happy//Happy1523.wav
D://Datasets//TRAINING//Happy//Happy1526.wav
D://Datasets//TRAINING//Happy//Happy1527.wav
D://Datasets//TRAINING//Happy//Happy1528.wav
D://Datasets//TRAINING//Happy//Happy1529.wav
D://Datasets//TRAINING//Happy//Happy1531.wav
D://Dataset

D://Datasets//TRAINING//Happy//Happy1739.wav
D://Datasets//TRAINING//Happy//Happy1740.wav
D://Datasets//TRAINING//Happy//Happy1741.wav
D://Datasets//TRAINING//Happy//Happy1743.wav
D://Datasets//TRAINING//Happy//Happy1744.wav
D://Datasets//TRAINING//Happy//Happy1745.wav
D://Datasets//TRAINING//Happy//Happy1747.wav
D://Datasets//TRAINING//Happy//Happy1748.wav
D://Datasets//TRAINING//Happy//Happy1749.wav
D://Datasets//TRAINING//Happy//Happy1750.wav
D://Datasets//TRAINING//Happy//Happy1751.wav
D://Datasets//TRAINING//Happy//Happy1752.wav
D://Datasets//TRAINING//Happy//Happy1753.wav
D://Datasets//TRAINING//Happy//Happy1754.wav
D://Datasets//TRAINING//Happy//Happy1755.wav
D://Datasets//TRAINING//Happy//Happy1756.wav
D://Datasets//TRAINING//Happy//Happy1757.wav
D://Datasets//TRAINING//Happy//Happy1758.wav
D://Datasets//TRAINING//Happy//Happy1759.wav
D://Datasets//TRAINING//Happy//Happy1761.wav
D://Datasets//TRAINING//Happy//Happy1762.wav
D://Datasets//TRAINING//Happy//Happy1764.wav
D://Datase

D://Datasets//TRAINING//Happy//Happy172.wav
D://Datasets//TRAINING//Happy//Happy174.wav
D://Datasets//TRAINING//Happy//Happy175.wav
D://Datasets//TRAINING//Happy//Happy176.wav
D://Datasets//TRAINING//Happy//Happy178.wav
D://Datasets//TRAINING//Happy//Happy180.wav
D://Datasets//TRAINING//Happy//Happy181.wav
D://Datasets//TRAINING//Happy//Happy182.wav
D://Datasets//TRAINING//Happy//Happy183.wav
D://Datasets//TRAINING//Happy//Happy184.wav
D://Datasets//TRAINING//Happy//Happy186.wav
D://Datasets//TRAINING//Happy//Happy187.wav
D://Datasets//TRAINING//Happy//Happy188.wav
D://Datasets//TRAINING//Happy//Happy189.wav
D://Datasets//TRAINING//Happy//Happy190.wav
D://Datasets//TRAINING//Happy//Happy191.wav
D://Datasets//TRAINING//Happy//Happy194.wav
D://Datasets//TRAINING//Happy//Happy196.wav
D://Datasets//TRAINING//Happy//Happy197.wav
D://Datasets//TRAINING//Happy//Happy198.wav
D://Datasets//TRAINING//Happy//Happy199.wav
D://Datasets//TRAINING//Happy//Happy200.wav
D://Datasets//TRAINING//Happy//H

D://Datasets//TRAINING//Happy_test//Happy567.wav
D://Datasets//TRAINING//Happy_test//Happy1437.wav
D://Datasets//TRAINING//Happy_test//Happy601.wav
D://Datasets//TRAINING//Happy_test//Happy1088.wav
D://Datasets//TRAINING//Happy_test//Happy1227.wav
D://Datasets//TRAINING//Happy_test//Happy673.wav
D://Datasets//TRAINING//Happy_test//Happy952.wav
D://Datasets//TRAINING//Happy_test//Happy657.wav
D://Datasets//TRAINING//Happy_test//Happy236.wav
D://Datasets//TRAINING//Happy_test//Happy489.wav
D://Datasets//TRAINING//Happy_test//Happy561.wav
D://Datasets//TRAINING//Happy_test//Happy1530.wav
D://Datasets//TRAINING//Happy_test//Happy527.wav
D://Datasets//TRAINING//Happy_test//Happy1683.wav
D://Datasets//TRAINING//Happy_test//Happy1200.wav
D://Datasets//TRAINING//Happy_test//Happy1566.wav
D://Datasets//TRAINING//Happy_test//Happy1079.wav
D://Datasets//TRAINING//Happy_test//Happy684.wav
D://Datasets//TRAINING//Happy_test//Happy1366.wav
D://Datasets//TRAINING//Happy_test//Happy575.wav
D://Dataset

D://Datasets//TRAINING//Happy_test//Happy1190.wav
D://Datasets//TRAINING//Happy_test//Happy1511.wav
D://Datasets//TRAINING//Happy_test//Happy1365.wav
D://Datasets//TRAINING//Happy_test//Happy1669.wav
D://Datasets//TRAINING//Happy_test//Happy225.wav
D://Datasets//TRAINING//Happy_test//Happy1780.wav
D://Datasets//TRAINING//Happy_test//Happy814.wav
D://Datasets//TRAINING//Happy_test//Happy1068.wav
D://Datasets//TRAINING//Happy_test//Happy265.wav
D://Datasets//TRAINING//Happy_test//Happy1252.wav
D://Datasets//TRAINING//Happy_test//Happy566.wav
D://Datasets//TRAINING//Happy_test//Happy1579.wav
D://Datasets//TRAINING//Happy_test//Happy1671.wav
D://Datasets//TRAINING//Happy_test//Happy1427.wav
D://Datasets//TRAINING//Happy_test//Happy1151.wav
D://Datasets//TRAINING//Happy_test//Happy1304.wav
D://Datasets//TRAINING//Happy_test//Happy1386.wav
D://Datasets//TRAINING//Happy_test//Happy462.wav
D://Datasets//TRAINING//Happy_test//Happy962.wav
D://Datasets//TRAINING//Happy_test//Happy1570.wav
D://Da

In [73]:
def change_amplitude(emotionfile, d1, newSoundFile, d2):
    
    if d1 <= d2:
        sound = AudioSegment.from_file(emotionfile) - np.random.randint(0, (6 * d2/d1 - 1))
        sound.export(newSoundFile, format='wav')  ### save the new generated file in a folder
    else:
        print('Invalid distance parameters. d1 should be <= d2.')

In [74]:
def change_amplitude_range(emotionfile, newSoundFile, threshold):
    #amount = np.random.randint(0, threshold)
    amount = random.uniform(0, threshold)
    #print('Deamplify ' + str(emotionfile) + ' by ' + str(amount) + ' db.')
    sound = AudioSegment.from_file(emotionfile) - amount
    sound.export(newSoundFile, format='wav')  ### save the new generated file in a folder
    return amount

In [77]:
'''
def deamplify_per_folder(directory):
    for file in os.listdir(directory):
        if file.endswith('.wav'):
            soundFile = directory + file
            newSoundFile = directory + 'deamplified_' + file
            change_amplitude_range(soundFile, newSoundFile, 12)
'''

"\ndef deamplify_per_folder(directory):\n    for file in os.listdir(directory):\n        if file.endswith('.wav'):\n            soundFile = directory + file\n            newSoundFile = directory + 'deamplified_' + file\n            change_amplitude_range(soundFile, newSoundFile, 12)\n"

In [78]:
'''
for directory in emotion_train_folders:
    deamplify_per_folder(directory)
    
for directory in emotion_test_folders:
    deamplify_per_folder(directory)
'''

'\nfor directory in emotion_train_folders:\n    deamplify_per_folder(directory)\n    \nfor directory in emotion_test_folders:\n    deamplify_per_folder(directory)\n'

In [79]:
def add_noise_per_file(emotionfile, bgnoise, newSoundFile):
    
    emotionsound = AudioSegment.from_wav(emotionfile)
    emotion_duration = emotionsound.duration_seconds * 1000
    noise = AudioSegment.from_wav(bgnoise)
    noise_duration = noise.duration_seconds * 1000
    
    threshold = noise_duration - emotion_duration
    
    if threshold > 0:
        overlay_start = np.random.randint(0, threshold)
    else:
        overlay_start = 0
        
    targeted_chunk = noise[overlay_start:overlay_start + emotion_duration]
    newSound = emotionsound.overlay(targeted_chunk, position=0)
    newSound=newSound[0:5000]
    newSound.export(newSoundFile, format='wav')  ### save the new generated file in a folder

In [80]:
'''
def add_noise_per_folder(directory, extension, noise_directory):
    for file in os.listdir(directory):
        if file.endswith(extension) and not file[1] == '_':
            for i in range(0, 3):
                soundFile = directory + file
                newSoundFile = directory + str(i) + '_' + file
                random_noise = noise_directory + random.choice(os.listdir(noise_directory))

                add_noise_per_file(soundFile, random_noise, newSoundFile)
                print(newSoundFile)
'''

"\ndef add_noise_per_folder(directory, extension, noise_directory):\n    for file in os.listdir(directory):\n        if file.endswith(extension) and not file[1] == '_':\n            for i in range(0, 3):\n                soundFile = directory + file\n                newSoundFile = directory + str(i) + '_' + file\n                random_noise = noise_directory + random.choice(os.listdir(noise_directory))\n\n                add_noise_per_file(soundFile, random_noise, newSoundFile)\n                print(newSoundFile)\n"

In [81]:
def add_noise_and_deamplify_per_folder(directory, extension, noise_directory):
    for file in os.listdir(directory):
        if file.endswith(extension) and not file[1] == '_':
            for i in range(0, 2):
                soundFile = directory + file
                amount = change_amplitude_range(soundFile, soundFile, 12)
                noise = random.choice(os.listdir(noise_directory))
                random_noise = noise_directory + noise
                newSoundFile = directory + 'deamp_' + str(amount) + '_noise_' + noise[:len(noise)-5] + '_' + file
                add_noise_per_file(soundFile, random_noise, newSoundFile)
                print(newSoundFile)

            

In [82]:
#add_noise_and_deamplify_per_folder('C://Users//yg9ca//Documents//Datasets//TEST//EMOTION//', 'wav', background_noise)

In [ ]:
add_noise_and_deamplify_per_folder(h_directory, 'wav', background_noise)
add_noise_and_deamplify_per_folder(a_directory, 'wav', background_noise)
add_noise_and_deamplify_per_folder(n_directory, 'wav', background_noise)
add_noise_and_deamplify_per_folder(s_directory, 'wav', background_noise)
add_noise_and_deamplify_per_folder(o_directory, 'wav', background_noise)

D://Datasets//TRAINING//Happy//deamp_10.604645203909602_noise_residential_area_3_Happy374.wav
D://Datasets//TRAINING//Happy//deamp_0.22589981673210424_noise_car_17_Happy374.wav
D://Datasets//TRAINING//Happy//deamp_1.8917498686684522_noise_grocery_store_75_Happy375.wav
D://Datasets//TRAINING//Happy//deamp_10.516106032872248_noise_car_80_Happy375.wav
D://Datasets//TRAINING//Happy//deamp_8.493747101296304_noise_library_58_Happy376.wav
D://Datasets//TRAINING//Happy//deamp_4.183201410177844_noise_park_52_Happy376.wav
D://Datasets//TRAINING//Happy//deamp_3.518053344460589_noise_park_9_Happy377.wav
D://Datasets//TRAINING//Happy//deamp_11.647594623427318_noise_office_88_Happy377.wav
D://Datasets//TRAINING//Happy//deamp_9.585284550338535_noise_park_9_Happy378.wav
D://Datasets//TRAINING//Happy//deamp_3.64967049215222_noise_a130_210_24_Happy378.wav
D://Datasets//TRAINING//Happy//deamp_9.369154435534004_noise_metro_station_71_Happy379.wav
D://Datasets//TRAINING//Happy//deamp_8.4794449868215_noise_

In [ ]:
add_noise_and_deamplify_per_folder(h_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(a_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(n_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(s_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(o_test, '.wav', background_noise_test)

In [ ]:
# AudioEffectsChain().reverb()(infile, outfile)